## Monte Carlo Evaluation
One advantage of Monte Carlo methods are that they doesn't require a model of the enviroment (e.g. the $p(s' | s, a)$ and $r(s,a,s')$ in Bellman equation).

In a nutshell, Monte Carlo methods collect statistics about $v(s)$ and $q(s, a)$. 

Take $v(s)$ as an example. This is defined as $\mathbb{E}(G_t | S_t = s)$ for any $t$. In the first-visit MC methods, we are only interested in the expected return from state $s$ after the very first visit, and we don't worry about the time index $t$. There are also every-visit MC methods. In Sutton's book, first-visit MC is the focus. 

Given state $s$, we think the return $G$ from any epsidoe is sampled from some distribution $p_s(G)$. We are not interested in the exact form of $p_s(G)$; in fact, that would require modeling of the enviroment. All we do is following the policy a number of times, and at the end of each episode, we have a sample $G$ from $p_s(G)$. Then $$v(s) = \mathbb{E}(G | s) \approx \frac{1}{N} \sum_i G_i$$

For example, suppose we have state sequence $S_0 S_1 S_2 S_1 S_3$ in one episode. $S_3$ is the terminating state and we get reward $x$ during the transition $S_1 \rightarrow S_3$, and zero reward in all other transitions. Suppose the discount facotr is $\gamma$. With this setup, $S_2$ gets a sample return $\gamma x$ and $S_1$ gets a sample return $\gamma^2 x$. Note only the first appearence of $S_1$ cares about the sample return. 

$q(s,a)$ is computed in the similar fashion (first-visit MC).

## Monte Carlo Control
